In [16]:
import pandas as pd
import numpy as np

movies = pd.read_csv("movieData/movies.csv")

ratings = pd.read_csv("movieData/ratings.csv")

In [17]:
movies.set_index("movieId", inplace=True)

counts = ratings["movieId"].value_counts()

movies["ratingsCount"] = counts

In [18]:
averageRatings = ratings.groupby("movieId").mean()["rating"]

movies["averageRatings"] = averageRatings

In [19]:
movies.sort_values(["ratingsCount", "averageRatings"], ascending=False)

minimumRatingsSubset = movies.query(f"ratingsCount >= 100").sort_values("averageRatings", ascending=False)

In [20]:
USER_1_POSITION_X = 3
USER_1_POSITION_Y = 7.5

USER_2_POSITION_X = 4
USER_2_POSITION_Y = 2

position_user__1 = np.array([USER_1_POSITION_X, USER_1_POSITION_Y])

position_user__2 = np.array([USER_2_POSITION_X, USER_2_POSITION_Y])

In [21]:
def calculateDistance(user_1, user_2):
  (delta_x, delta_y) = user_1 - user_2
  
  return (delta_x ** 2 + delta_y ** 2) ** (0.5)


distance = calculateDistance(position_user__1, position_user__2)
print(distance)

5.5901699437494745
